# Лабораторная работа №4. Метод опорных векторов

Выполнили: Рутин Василий, Гордеев Станислав

Все лабораторные работы доступны по адресу: https://github.com/Omenstudio/ML_IFMO_LABS

## 1. Постановка задачи

1. Прочитать теоретическую часть
2. Описать структуру исходных данных для своего набора:<br>
	a. общие характеристики массива данных: предметная область, количество записей<br>
	b. входные параметры: названия и типы<br>
    c. выходной класс: название и значения<br>
3. Осуществить ряд экспериментов по классификации, используя SVM с различными параметрами (функции ядра и пр.), и занести результаты в сравнительную таблицу.
4. Выбрать оптимальные параметры и сформировать вывод о применимости метода опорных векторов к задаче классификации для своего набора данных.

## 2. Исходные данные

Задача классификации: определение наличия заболевания (диабет) у женщин.<br>
Датасет доступен по адресу: https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

Количество записей: 768<br>
Количество атрибутов: 8 плюс класс.<br>
Все атрибуты представлены числовыми значениями:
1. Количество беременностей (шт.)
2. Концентрация плазмы глюкозы (ммоль/л)
3. Диастолическое давление (мм. рт. столба)
4. Толщина кожной складки (мм.)
5. Содержание инсулина (мкЕд/мл)
6. Индекс массы тела (кг/м2)
7. Предрасположенность по родословной линии (Diabetes pedigree function)
8. Возраст (лет)
9. Класс: наличие диабета (0, 1)

## 3. Ход работы

Загружаем датасет.

In [1]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


def load_dataset(split_ratio=None, normalize=True):
    dataset_full = np.loadtxt(open("../pima-indians-diabetes.data.csv", "r"), delimiter=",", skiprows=0, dtype=np.float64)
    inputs = dataset_full[:, :-1]
    outputs = dataset_full[:, -1]
    outputs = outputs.astype(np.int64, copy=False)
    if split_ratio is None:
        return inputs, outputs, None, None
    inputs_train, inputs_test, outputs_train, outputs_test \
        = train_test_split(inputs, outputs, test_size=split_ratio, random_state=random.randint(0, 1000))
    if normalize:
        std_scale = preprocessing.StandardScaler().fit(inputs_train)
        inputs_train = std_scale.transform(inputs_train)
        inputs_test = std_scale.transform(inputs_test)
    return inputs_train, outputs_train, inputs_test, outputs_test

Создаем различные вариации параметров, которые будем тестировать.

In [2]:
import numpy as np
from sklearn.svm import SVC

# Чтение датасета
inputs_train, outputs_train, inputs_test, outputs_test = load_dataset(split_ratio=.4, normalize=True)
#  Варианты
kernel_types = ['linear', 'poly', 'rbf', 'sigmoid']
penalties = np.arange(.5, 10, 0.5)
degrees = np.arange(1, 10, 1)
gammas = np.arange(0.05, 1, 0.05)
coefs0 = np.arange(0, 1, 0.1)
probabilities = [False, True]
shrinkings = [False, True]
tols = [1e-1, 1e-2, 1e-3, 1e-4]

Тестируем различные параметры:

In [3]:
for value in kernel_types:
    clf = SVC(kernel=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('kernel_types: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

kernel_types: linear, accuracy: 0.779220779
kernel_types: poly, accuracy: 0.750000000
kernel_types: rbf, accuracy: 0.772727273
kernel_types: sigmoid, accuracy: 0.698051948
-------------


На выбранном датасете наиболее эффективным оказывается линейная функция ядра ПРИ ПРОЧИХ РАВНЫХ. Радиальная базисная чуть хуже (разница в ~0,6%). Полиномиальная и сигмоидная показывают результаты хуже, но они имеют множество настриваемых параметров, нужно исследовать дальше.

In [4]:
for value in penalties:
    clf = SVC(C=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('penalties: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

penalties: 0.5, accuracy: 0.775974026
penalties: 1.0, accuracy: 0.772727273
penalties: 1.5, accuracy: 0.772727273
penalties: 2.0, accuracy: 0.766233766
penalties: 2.5, accuracy: 0.766233766
penalties: 3.0, accuracy: 0.750000000
penalties: 3.5, accuracy: 0.743506494
penalties: 4.0, accuracy: 0.746753247
penalties: 4.5, accuracy: 0.743506494
penalties: 5.0, accuracy: 0.746753247
penalties: 5.5, accuracy: 0.750000000
penalties: 6.0, accuracy: 0.753246753
penalties: 6.5, accuracy: 0.750000000
penalties: 7.0, accuracy: 0.750000000
penalties: 7.5, accuracy: 0.750000000
penalties: 8.0, accuracy: 0.753246753
penalties: 8.5, accuracy: 0.750000000
penalties: 9.0, accuracy: 0.746753247
penalties: 9.5, accuracy: 0.746753247
-------------


Легко заметить: чем больше параметр penalty, тем меньше точность.

Протестируем параметр degree, который используется только при полиномиальной функции ядра. 

In [5]:
for value in degrees:
    clf = SVC(kernel='poly', degree=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('degrees: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

degrees: 1, accuracy: 0.772727273
degrees: 2, accuracy: 0.717532468
degrees: 3, accuracy: 0.750000000
degrees: 4, accuracy: 0.711038961
degrees: 5, accuracy: 0.737012987
degrees: 6, accuracy: 0.688311688
degrees: 7, accuracy: 0.737012987
degrees: 8, accuracy: 0.681818182
degrees: 9, accuracy: 0.714285714
-------------


Видно, что при degree=1, точность составляет 77.27%, это уже сопоставимо с результатами линейной и радиальной базисной функциями на этом датасете.

Параметр gamma используется при функциях ядра poly, sigmoid и rbf. Давайте посмотрим, как его отклонение влияет на точность в каждой из функций ядра.

In [6]:
for value in gammas:
    clf = SVC(kernel='poly', gamma=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('gammas: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

gammas: 0.05, accuracy: 0.707792208
gammas: 0.1, accuracy: 0.750000000
gammas: 0.15000000000000002, accuracy: 0.743506494
gammas: 0.2, accuracy: 0.740259740
gammas: 0.25, accuracy: 0.730519481
gammas: 0.3, accuracy: 0.717532468
gammas: 0.35000000000000003, accuracy: 0.704545455
gammas: 0.4, accuracy: 0.701298701
gammas: 0.45, accuracy: 0.707792208
gammas: 0.5, accuracy: 0.711038961
gammas: 0.55, accuracy: 0.698051948
gammas: 0.6000000000000001, accuracy: 0.698051948
gammas: 0.6500000000000001, accuracy: 0.691558442
gammas: 0.7000000000000001, accuracy: 0.694805195
gammas: 0.7500000000000001, accuracy: 0.701298701
gammas: 0.8, accuracy: 0.685064935
gammas: 0.8500000000000001, accuracy: 0.688311688
gammas: 0.9000000000000001, accuracy: 0.688311688
gammas: 0.9500000000000001, accuracy: 0.691558442
-------------


In [12]:
for value in gammas:
    clf = SVC(kernel='rbf', gamma=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('gammas: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

gammas: 0.05, accuracy: 0.788961039
gammas: 0.1, accuracy: 0.772727273
gammas: 0.15000000000000002, accuracy: 0.775974026
gammas: 0.2, accuracy: 0.766233766
gammas: 0.25, accuracy: 0.762987013
gammas: 0.3, accuracy: 0.759740260
gammas: 0.35000000000000003, accuracy: 0.756493506
gammas: 0.4, accuracy: 0.756493506
gammas: 0.45, accuracy: 0.753246753
gammas: 0.5, accuracy: 0.753246753
gammas: 0.55, accuracy: 0.746753247
gammas: 0.6000000000000001, accuracy: 0.740259740
gammas: 0.6500000000000001, accuracy: 0.737012987
gammas: 0.7000000000000001, accuracy: 0.740259740
gammas: 0.7500000000000001, accuracy: 0.743506494
gammas: 0.8, accuracy: 0.746753247
gammas: 0.8500000000000001, accuracy: 0.743506494
gammas: 0.9000000000000001, accuracy: 0.740259740
gammas: 0.9500000000000001, accuracy: 0.727272727
-------------


In [13]:
for value in gammas:
    clf = SVC(kernel='sigmoid', gamma=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('gammas: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

gammas: 0.05, accuracy: 0.769480519
gammas: 0.1, accuracy: 0.704545455
gammas: 0.15000000000000002, accuracy: 0.704545455
gammas: 0.2, accuracy: 0.698051948
gammas: 0.25, accuracy: 0.668831169
gammas: 0.3, accuracy: 0.688311688
gammas: 0.35000000000000003, accuracy: 0.691558442
gammas: 0.4, accuracy: 0.694805195
gammas: 0.45, accuracy: 0.685064935
gammas: 0.5, accuracy: 0.675324675
gammas: 0.55, accuracy: 0.681818182
gammas: 0.6000000000000001, accuracy: 0.685064935
gammas: 0.6500000000000001, accuracy: 0.681818182
gammas: 0.7000000000000001, accuracy: 0.681818182
gammas: 0.7500000000000001, accuracy: 0.678571429
gammas: 0.8, accuracy: 0.685064935
gammas: 0.8500000000000001, accuracy: 0.675324675
gammas: 0.9000000000000001, accuracy: 0.691558442
gammas: 0.9500000000000001, accuracy: 0.685064935
-------------


Заметим, что при увеличении параметра гаммы, точность снижается во всех 3 тестах. rbf показал самую большую точность (78.8%), которая превзошла результат линейной функции ядра (77.9%) из первого теста!

In [7]:
for value in coefs0:
    clf = SVC(kernel='poly', coef0=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('coefs0: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

coefs0: 0.0, accuracy: 0.750000000
coefs0: 0.1, accuracy: 0.756493506
coefs0: 0.2, accuracy: 0.772727273
coefs0: 0.30000000000000004, accuracy: 0.766233766
coefs0: 0.4, accuracy: 0.762987013
coefs0: 0.5, accuracy: 0.762987013
coefs0: 0.6000000000000001, accuracy: 0.759740260
coefs0: 0.7000000000000001, accuracy: 0.759740260
coefs0: 0.8, accuracy: 0.759740260
coefs0: 0.9, accuracy: 0.756493506


Самым эффективным из протестированных является параметр coef0=0.3

In [14]:
for value in probabilities:
    clf = SVC(probability=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('probabilities: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')
for value in shrinkings:
    clf = SVC(shrinking=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('shrinkings: {}, accuracy: {:.9f}'.format(value, accuracy))
print('-------------')

probabilities: False, accuracy: 0.772727273
probabilities: True, accuracy: 0.772727273
-------------
shrinkings: False, accuracy: 0.772727273
shrinkings: True, accuracy: 0.772727273
-------------


Изменения параметров shrinking и probability, никак не влияет на результат.

In [15]:
for value in tols:
    clf = SVC(tol=value)
    clf.fit(inputs_train, outputs_train)
    accuracy = clf.score(inputs_test, outputs_test)
    print('tols: {}, accuracy: {:.9f}'.format(value, accuracy))

tols: 0.1, accuracy: 0.772727273
tols: 0.01, accuracy: 0.772727273
tols: 0.001, accuracy: 0.772727273
tols: 0.0001, accuracy: 0.772727273


Изменение параметра tol так же никак не влияет на точность.

Попробуем запустить SVM, задав множество параметров, а не один.

In [39]:
clf = SVC(C=3.1, kernel='rbf', degree=1, gamma=0.05, coef0=1.1)
clf.fit(inputs_train, outputs_train)
accuracy = clf.score(inputs_test, outputs_test)
print('{}'.format(clf))
print('Accuracy: {:.9f}'.format(accuracy))

SVC(C=3.1, cache_size=200, class_weight=None, coef0=1.1,
  decision_function_shape=None, degree=1, gamma=0.05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Accuracy: 0.798701299


Путем нехитрых манипуляций получили точность 79.87%, что является рекордом на проведенных тестах.

## 4. Заключение

В ходе лабораторной работы проведены эксперименты и сделаны выводы для текущего набора данных:<br>
1. При прочих равных наиболее эффективной оказывается линейная функция ядра.
2. При настройке параметров, радиально базисная функция превосходит по точности линейную функцию ядра.
3. При прочих равных чем больше параметр penalty, тем ниже точность.
4. При увеличении параметра gamma, точность снижается.
5. При прочих равных coef0=0.3 показал лучшие результаты.
6. Изменения параметров shrinking, probability и tol никак не влияет на точность, однако probability лучше устанавливать False из-за возрастания времени работы при значении True.

Очень важной является фраза "при прочих равных". Самый последний тест с попыткой максимизировать точность показал высокие результаты при параметрах penalty(C)=3.1 и coef0=1.1, что не соотносится с выводами 3, 5. Соответственно, либо а) выборка "неудачно" поделилась, а тесты следует проводить с перекрестной проверкой (KFold), либо б) влияние параметров сильно зависит друг от друга, в таком случае проводить эксперименты, оценивая влияение каждого по-отдельности, некорректно.